<a href="https://colab.research.google.com/github/Rocco000/OncoVision/blob/main/Scripts/ModelArchitecture2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch.nn.modules.pooling import MaxPool2d
import torch.nn as nn
import torch.nn.functional as F
import torchvision

from google.colab import drive
drive.mount('/content/drive') #Connect to Google Drive

def choose_layer(layer_type, channels=8):
  match layer_type:
    case 1:
      return nn.Conv2d(in_channels=channels, out_channels=128, kernel_size=3, stride=1, padding=1, bias=True)
    case 2:
      return nn.Conv2d(in_channels=channels, out_channels=64, kernel_size=3, stride=1, padding=1, bias=True)
    case 3:
      return nn.Conv2d(in_channels=channels, out_channels=32, kernel_size=3, stride=1, padding=1, bias=True)
    case 4:
      return nn.Conv2d(in_channels=channels, out_channels=16, kernel_size=3, stride=1, padding=1, bias=True)
    case 5:
      return nn.Conv2d(in_channels=channels, out_channels=8, kernel_size=3, stride=1, padding=1, bias=True)
    case 6:
      return nn.MaxPool2d(kernel_size=3, stride=3)
    case 7:
      return nn.MaxPool2d(kernel_size=2, stride=2)
    case 8:
      return nn.AvgPool2d(kernel_size=3, stride=3)
    case 9:
      return nn.AvgPool2d(kernel_size=2, stride=2)
    case 10:
      return nn.Dropout2d(p=0.2)
    case 11:
      return nn.BatchNorm2d(channels)
    case 12:
      return nn.ReLU()
    case 13:
      return nn.LeakyReLU()
    case _:
      print("Value error")
      return None

def size_nn_linear_calculator(layer_type, width, height, channels):
  """
    1: cnn-128;
    2: cnn-64;
    3: cnn-32;
    4: cnn-16;
    5: cnn-8;
    6: max-3;
    7: max-2;
    8: avg-3;
    9: avg-2;
    10: dropout;
    11: batch-norm;
    12: ReLu;
    13: LeakyReLu
  """
  new_width=new_height=size = 0
  match layer_type:
    case 1:
      #cnn-128
      new_width = ((width - 3 + 2)/1)+1 # [(W-F+2P)/S]+1
      new_height = ((height - 3 + 2)/1)+1 # [(H-F+2P)/S]+1
      size = new_width * new_height * 128
    case 2:
      #cnn-64
      new_width = ((width - 3 + 2)/1)+1 # [(W-F+2P)/S]+1
      new_height = ((height - 3 + 2)/1)+1 # [(H-F+2P)/S]+1
      size = new_width * new_height * 64
    case 3:
      #cnn-32
      new_width = ((width - 3 + 2)/1)+1 # [(W-F+2P)/S]+1
      new_height = ((height - 3 + 2)/1)+1 # [(H-F+2P)/S]+1
      size = new_width * new_height * 32
    case 4:
      #cnn-16
      new_width = ((width - 3 + 2)/1)+1 # [(W-F+2P)/S]+1
      new_height = ((height - 3 + 2)/1)+1 # [(H-F+2P)/S]+1
      size = new_width * new_height * 16
    case 5:
      #cnn-8
      new_width = ((width - 3 + 2)/1)+1 # [(W-F+2P)/S]+1
      new_height = ((height - 3 + 2)/1)+1 # [(H-F+2P)/S]+1
      size = new_width * new_height * 8
    case 6:
      #max-3
      frac_value = int((width-3)/3)
      new_width = frac_value+1 # Wout = [(Win - F)/S]+1
      frac_value = int((height-3)/3)
      new_height = frac_value+1 # Hout = [(Hin - F)/S]+1
      size = new_width * new_height * channels
    case 7:
      #max-2
      frac_value = int((width-2)/2)
      new_width = frac_value+1
      frac_value = int((height-2)/2)
      new_height = frac_value+1
      size = new_width * new_height * channels
    case 8:
      #avg-3
      frac_value = int((width-3)/3)
      new_width = frac_value+1
      frac_value = int((height-3)/3)
      new_height = frac_value+1
      size = new_width * new_height * channels
    case 9:
      #avg-2
      frac_value = int((width-2)/2)
      new_width = frac_value+1
      frac_value = int((height-2)/2)
      new_height = frac_value+1
      size = new_width * new_height * channels
    case _:
      print("Value error!")

  return new_width, new_height, size



class ConvModel2(nn.Module):
  def __init__(self, architecture, size):
    super(ConvModel2, self).__init__()
    self.layer0 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3, stride=1, padding=1, bias=True)
    self.layer1 = None
    self.layer2 = None
    self.layer3 = None
    self.layer4 = None
    self.layer5 = None
    self.layer6 = None
    self.layer7 = None
    self.layer8 = None
    self.layer9 = None
    self.layer10 = None
    self.layer11 = None
    self.layer12 = None
    self.layer13 = None
    self.layer14 = None
    self.layer15 = None
    self.layer16 = None
    self.flatter = nn.Flatten()

    #Build the netwrok
    for i in range(len(architecture)):
      match i:
        case 0:
          self.layer1 = choose_layer(architecture[i])
        case 1:
          #layer2
          if (architecture[i]>=1 and architecture[i]<=5) or architecture[i]==11: #if it is a convolutional layer or a batchnorm layer, we must set its in_channels value to the out_channels of the last convolutional layer
            if isinstance(self.layer1, nn.Conv2d):
              self.layer2 = choose_layer(architecture[i], self.layer1.out_channels)
            else:
              self.layer2 = choose_layer(architecture[i])
          else:
            self.layer2 = choose_layer(architecture[i])

        case 2:
          #layer3
          if (architecture[i]>=1 and architecture[i]<=5) or architecture[i]==11: #if it is a convolutional layer or a batchnorm layer, we must set its in_channels value to the out_channels of the last convolutional layer
            if isinstance(self.layer2, nn.Conv2d):
              self.layer3 = choose_layer(architecture[i], self.layer2.out_channels)
            elif isinstance(self.layer1, nn.Conv2d):
              self.layer3 = choose_layer(architecture[i], self.layer1.out_channels)
            else:
              self.layer3 = choose_layer(architecture[i])
          else:
            self.layer3 = choose_layer(architecture[i])

        case 3:
          #layer4
          if (architecture[i]>=1 and architecture[i]<=5) or architecture[i]==11: #if it is a convolutional layer or a batchnorm layer, we must set its in_channels value to the out_channels of the last convolutional layer
            if isinstance(self.layer3, nn.Conv2d):
              self.layer4 = choose_layer(architecture[i], self.layer3.out_channels)
            elif isinstance(self.layer2, nn.Conv2d):
              self.layer4 = choose_layer(architecture[i], self.layer2.out_channels)
            elif isinstance(self.layer1, nn.Conv2d):
              self.layer4 = choose_layer(architecture[i], self.layer1.out_channels)
            else:
              self.layer4 = choose_layer(architecture[i])
          else:
            self.layer4 = choose_layer(architecture[i])
        case 4:
          #layer5
          if (architecture[i]>=1 and architecture[i]<=5) or architecture[i]==11: #if it is a convolutional layer or a batchnorm layer, we must set its in_channels value to the out_channels of the last convolutional layer
            if isinstance(self.layer4, nn.Conv2d):
              self.layer5 = choose_layer(architecture[i], self.layer4.out_channels)
            elif isinstance(self.layer3, nn.Conv2d):
              self.layer5 = choose_layer(architecture[i], self.layer3.out_channels)
            elif isinstance(self.layer2, nn.Conv2d):
              self.layer5 = choose_layer(architecture[i], self.layer2.out_channels)
            elif isinstance(self.layer1, nn.Conv2d):
              self.layer5 = choose_layer(architecture[i], self.layer1.out_channels)
            else:
              self.layer5 = choose_layer(architecture[i])
          else:
            self.layer5 = choose_layer(architecture[i])
        case 5:
          #layer6
          if (architecture[i]>=1 and architecture[i]<=5) or architecture[i]==11: #if it is a convolutional layer or a batchnorm layer, we must set its in_channels value to the out_channels of the last convolutional layer
            if isinstance(self.layer5, nn.Conv2d):
              self.layer6 = choose_layer(architecture[i], self.layer5.out_channels)
            elif isinstance(self.layer4, nn.Conv2d):
              self.layer6 = choose_layer(architecture[i], self.layer4.out_channels)
            elif isinstance(self.layer3, nn.Conv2d):
              self.layer6 = choose_layer(architecture[i], self.layer3.out_channels)
            elif isinstance(self.layer2, nn.Conv2d):
              self.layer6 = choose_layer(architecture[i], self.layer2.out_channels)
            elif isinstance(self.layer1, nn.Conv2d):
              self.layer6 = choose_layer(architecture[i], self.layer1.out_channels)
            else:
              self.layer6 = choose_layer(architecture[i])
          else:
            self.layer6 = choose_layer(architecture[i])
        case 6:
          #layer7
          if (architecture[i]>=1 and architecture[i]<=5) or architecture[i]==11: #if it is a convolutional layer or a batchnorm layer, we must set its in_channels value to the out_channels of the last convolutional layer
            if isinstance(self.layer6, nn.Conv2d):
              self.layer7 = choose_layer(architecture[i], self.layer6.out_channels)
            elif isinstance(self.layer5, nn.Conv2d):
              self.layer7 = choose_layer(architecture[i], self.layer5.out_channels)
            elif isinstance(self.layer4, nn.Conv2d):
              self.layer7 = choose_layer(architecture[i], self.layer4.out_channels)
            elif isinstance(self.layer3, nn.Conv2d):
              self.layer7 = choose_layer(architecture[i], self.layer3.out_channels)
            elif isinstance(self.layer2, nn.Conv2d):
              self.layer7 = choose_layer(architecture[i], self.layer2.out_channels)
            elif isinstance(self.layer1, nn.Conv2d):
              self.layer7 = choose_layer(architecture[i], self.layer1.out_channels)
            else:
              self.layer7 = choose_layer(architecture[i])
          else:
            self.layer7 = choose_layer(architecture[i])
        case 7:
          #layer8
          if (architecture[i]>=1 and architecture[i]<=5) or architecture[i]==11: #if it is a convolutional layer or a batchnorm layer, we must set its in_channels value to the out_channels of the last convolutional layer
            if isinstance(self.layer7, nn.Conv2d):
              self.layer8 = choose_layer(architecture[i], self.layer7.out_channels)
            elif isinstance(self.layer6, nn.Conv2d):
              self.layer8 = choose_layer(architecture[i], self.layer6.out_channels)
            elif isinstance(self.layer5, nn.Conv2d):
              self.layer8 = choose_layer(architecture[i], self.layer5.out_channels)
            elif isinstance(self.layer4, nn.Conv2d):
              self.layer8 = choose_layer(architecture[i], self.layer4.out_channels)
            elif isinstance(self.layer3, nn.Conv2d):
              self.layer8 = choose_layer(architecture[i], self.layer3.out_channels)
            elif isinstance(self.layer2, nn.Conv2d):
              self.layer8 = choose_layer(architecture[i], self.layer2.out_channels)
            elif isinstance(self.layer1, nn.Conv2d):
              self.layer8 = choose_layer(architecture[i], self.layer1.out_channels)
            else:
              self.layer8 = choose_layer(architecture[i])
          else:
            self.layer8 = choose_layer(architecture[i])
        case 8:
          #layer9
          if (architecture[i]>=1 and architecture[i]<=5) or architecture[i]==11: #if it is a convolutional layer or a batchnorm layer, we must set its in_channels value to the out_channels of the last convolutional layer
            if isinstance(self.layer8, nn.Conv2d):
              self.layer9 = choose_layer(architecture[i], self.layer8.out_channels)
            elif isinstance(self.layer7, nn.Conv2d):
              self.layer9 = choose_layer(architecture[i], self.layer7.out_channels)
            elif isinstance(self.layer6, nn.Conv2d):
              self.layer9 = choose_layer(architecture[i], self.layer6.out_channels)
            elif isinstance(self.layer5, nn.Conv2d):
              self.layer9 = choose_layer(architecture[i], self.layer5.out_channels)
            elif isinstance(self.layer4, nn.Conv2d):
              self.layer9 = choose_layer(architecture[i], self.layer4.out_channels)
            elif isinstance(self.layer3, nn.Conv2d):
              self.layer9 = choose_layer(architecture[i], self.layer3.out_channels)
            elif isinstance(self.layer2, nn.Conv2d):
              self.layer9 = choose_layer(architecture[i], self.layer2.out_channels)
            elif isinstance(self.layer1, nn.Conv2d):
              self.layer9 = choose_layer(architecture[i], self.layer1.out_channels)
            else:
              self.layer9 = choose_layer(architecture[i])
          else:
            self.layer9 = choose_layer(architecture[i])
        case 9:
          #layer10
          if (architecture[i]>=1 and architecture[i]<=5) or architecture[i]==11: #if it is a convolutional layer or a batchnorm layer, we must set its in_channels value to the out_channels of the last convolutional layer
            if isinstance(self.layer9, nn.Conv2d):
              self.layer10 = choose_layer(architecture[i], self.layer9.out_channels)
            elif isinstance(self.layer8, nn.Conv2d):
              self.layer10 = choose_layer(architecture[i], self.layer8.out_channels)
            elif isinstance(self.layer7, nn.Conv2d):
              self.layer10 = choose_layer(architecture[i], self.layer7.out_channels)
            elif isinstance(self.layer6, nn.Conv2d):
              self.layer10 = choose_layer(architecture[i], self.layer6.out_channels)
            elif isinstance(self.layer5, nn.Conv2d):
              self.layer10 = choose_layer(architecture[i], self.layer5.out_channels)
            elif isinstance(self.layer4, nn.Conv2d):
              self.layer10 = choose_layer(architecture[i], self.layer4.out_channels)
            elif isinstance(self.layer3, nn.Conv2d):
              self.layer10 = choose_layer(architecture[i], self.layer3.out_channels)
            elif isinstance(self.layer2, nn.Conv2d):
              self.layer10 = choose_layer(architecture[i], self.layer2.out_channels)
            elif isinstance(self.layer1, nn.Conv2d):
              self.layer10 = choose_layer(architecture[i], self.layer1.out_channels)
            else:
              self.layer10 = choose_layer(architecture[i])
          else:
            self.layer10 = choose_layer(architecture[i])
        case 10:
          #layer11
          if (architecture[i]>=1 and architecture[i]<=5) or architecture[i]==11: #if it is a convolutional layer or a batchnorm layer, we must set its in_channels value to the out_channels of the last convolutional layer
            if isinstance(self.layer10, nn.Conv2d):
              self.layer11 = choose_layer(architecture[i], self.layer10.out_channels)
            elif isinstance(self.layer9, nn.Conv2d):
              self.layer11 = choose_layer(architecture[i], self.layer9.out_channels)
            elif isinstance(self.layer8, nn.Conv2d):
              self.layer11 = choose_layer(architecture[i], self.layer8.out_channels)
            elif isinstance(self.layer7, nn.Conv2d):
              self.layer11 = choose_layer(architecture[i], self.layer7.out_channels)
            elif isinstance(self.layer6, nn.Conv2d):
              self.layer11 = choose_layer(architecture[i], self.layer6.out_channels)
            elif isinstance(self.layer5, nn.Conv2d):
              self.layer11 = choose_layer(architecture[i], self.layer5.out_channels)
            elif isinstance(self.layer4, nn.Conv2d):
              self.layer11 = choose_layer(architecture[i], self.layer4.out_channels)
            elif isinstance(self.layer3, nn.Conv2d):
              self.layer11 = choose_layer(architecture[i], self.layer3.out_channels)
            elif isinstance(self.layer2, nn.Conv2d):
              self.layer11 = choose_layer(architecture[i], self.layer2.out_channels)
            elif isinstance(self.layer1, nn.Conv2d):
              self.layer11 = choose_layer(architecture[i], self.layer1.out_channels)
            else:
              self.layer11 = choose_layer(architecture[i])
          else:
            self.layer11 = choose_layer(architecture[i])
        case 11:
          #layer12
          if (architecture[i]>=1 and architecture[i]<=5) or architecture[i]==11: #if it is a convolutional layer or a batchnorm layer, we must set its in_channels value to the out_channels of the last convolutional layer
            if isinstance(self.layer11, nn.Conv2d):
              self.layer12 = choose_layer(architecture[i], self.layer11.out_channels)
            elif isinstance(self.layer10, nn.Conv2d):
              self.layer12 = choose_layer(architecture[i], self.layer10.out_channels)
            elif isinstance(self.layer9, nn.Conv2d):
              self.layer12 = choose_layer(architecture[i], self.layer9.out_channels)
            elif isinstance(self.layer8, nn.Conv2d):
              self.layer12 = choose_layer(architecture[i], self.layer8.out_channels)
            elif isinstance(self.layer7, nn.Conv2d):
              self.layer12 = choose_layer(architecture[i], self.layer7.out_channels)
            elif isinstance(self.layer6, nn.Conv2d):
              self.layer12 = choose_layer(architecture[i], self.layer6.out_channels)
            elif isinstance(self.layer5, nn.Conv2d):
              self.layer12 = choose_layer(architecture[i], self.layer5.out_channels)
            elif isinstance(self.layer4, nn.Conv2d):
              self.layer12 = choose_layer(architecture[i], self.layer4.out_channels)
            elif isinstance(self.layer3, nn.Conv2d):
              self.layer12 = choose_layer(architecture[i], self.layer3.out_channels)
            elif isinstance(self.layer2, nn.Conv2d):
              self.layer12 = choose_layer(architecture[i], self.layer2.out_channels)
            elif isinstance(self.layer1, nn.Conv2d):
              self.layer12 = choose_layer(architecture[i], self.layer1.out_channels)
            else:
              self.layer12 = choose_layer(architecture[i])
          else:
            self.layer12 = choose_layer(architecture[i])
        case 12:
          #layer13
          if (architecture[i]>=1 and architecture[i]<=5) or architecture[i]==11: #if it is a convolutional layer or a batchnorm layer, we must set its in_channels value to the out_channels of the last convolutional layer
            if isinstance(self.layer12, nn.Conv2d):
              self.layer13 = choose_layer(architecture[i], self.layer12.out_channels)
            elif isinstance(self.layer11, nn.Conv2d):
              self.layer13 = choose_layer(architecture[i], self.layer11.out_channels)
            elif isinstance(self.layer10, nn.Conv2d):
              self.layer13 = choose_layer(architecture[i], self.layer10.out_channels)
            elif isinstance(self.layer9, nn.Conv2d):
              self.layer13 = choose_layer(architecture[i], self.layer9.out_channels)
            elif isinstance(self.layer8, nn.Conv2d):
              self.layer13 = choose_layer(architecture[i], self.layer8.out_channels)
            elif isinstance(self.layer7, nn.Conv2d):
              self.layer13 = choose_layer(architecture[i], self.layer7.out_channels)
            elif isinstance(self.layer6, nn.Conv2d):
              self.layer13 = choose_layer(architecture[i], self.layer6.out_channels)
            elif isinstance(self.layer5, nn.Conv2d):
              self.layer13 = choose_layer(architecture[i], self.layer5.out_channels)
            elif isinstance(self.layer4, nn.Conv2d):
              self.layer13 = choose_layer(architecture[i], self.layer4.out_channels)
            elif isinstance(self.layer3, nn.Conv2d):
              self.layer13 = choose_layer(architecture[i], self.layer3.out_channels)
            elif isinstance(self.layer2, nn.Conv2d):
              self.layer13 = choose_layer(architecture[i], self.layer2.out_channels)
            elif isinstance(self.layer1, nn.Conv2d):
              self.layer13 = choose_layer(architecture[i], self.layer1.out_channels)
            else:
              self.layer13 = choose_layer(architecture[i])
          else:
            self.layer13 = choose_layer(architecture[i])
        case 13:
          #layer14
          if (architecture[i]>=1 and architecture[i]<=5) or architecture[i]==11: #if it is a convolutional layer or a batchnorm layer, we must set its in_channels value to the out_channels of the last convolutional layer
            if isinstance(self.layer13, nn.Conv2d):
              self.layer14 = choose_layer(architecture[i], self.layer13.out_channels)
            elif isinstance(self.layer12, nn.Conv2d):
              self.layer14 = choose_layer(architecture[i], self.layer12.out_channels)
            elif isinstance(self.layer11, nn.Conv2d):
              self.layer14 = choose_layer(architecture[i], self.layer11.out_channels)
            elif isinstance(self.layer10, nn.Conv2d):
              self.layer14 = choose_layer(architecture[i], self.layer10.out_channels)
            elif isinstance(self.layer9, nn.Conv2d):
              self.layer14 = choose_layer(architecture[i], self.layer9.out_channels)
            elif isinstance(self.layer8, nn.Conv2d):
              self.layer14 = choose_layer(architecture[i], self.layer8.out_channels)
            elif isinstance(self.layer7, nn.Conv2d):
              self.layer14 = choose_layer(architecture[i], self.layer7.out_channels)
            elif isinstance(self.layer6, nn.Conv2d):
              self.layer14 = choose_layer(architecture[i], self.layer6.out_channels)
            elif isinstance(self.layer5, nn.Conv2d):
              self.layer14 = choose_layer(architecture[i], self.layer5.out_channels)
            elif isinstance(self.layer4, nn.Conv2d):
              self.layer14 = choose_layer(architecture[i], self.layer4.out_channels)
            elif isinstance(self.layer3, nn.Conv2d):
              self.layer14 = choose_layer(architecture[i], self.layer3.out_channels)
            elif isinstance(self.layer2, nn.Conv2d):
              self.layer14 = choose_layer(architecture[i], self.layer2.out_channels)
            elif isinstance(self.layer1, nn.Conv2d):
              self.layer14 = choose_layer(architecture[i], self.layer1.out_channels)
            else:
              self.layer14 = choose_layer(architecture[i])
          else:
            self.layer14 = choose_layer(architecture[i])
        case 14:
          #layer15
          if (architecture[i]>=1 and architecture[i]<=5) or architecture[i]==11: #if it is a convolutional layer or a batchnorm layer, we must set its in_channels value to the out_channels of the last convolutional layer
            if isinstance(self.layer14, nn.Conv2d):
              self.layer15 = choose_layer(architecture[i], self.layer14.out_channels)
            elif isinstance(self.layer13, nn.Conv2d):
              self.layer15 = choose_layer(architecture[i], self.layer13.out_channels)
            elif isinstance(self.layer12, nn.Conv2d):
              self.layer15 = choose_layer(architecture[i], self.layer12.out_channels)
            elif isinstance(self.layer11, nn.Conv2d):
              self.layer15 = choose_layer(architecture[i], self.layer11.out_channels)
            elif isinstance(self.layer10, nn.Conv2d):
              self.layer15 = choose_layer(architecture[i], self.layer10.out_channels)
            elif isinstance(self.layer9, nn.Conv2d):
              self.layer15 = choose_layer(architecture[i], self.layer9.out_channels)
            elif isinstance(self.layer8, nn.Conv2d):
              self.layer15 = choose_layer(architecture[i], self.layer8.out_channels)
            elif isinstance(self.layer7, nn.Conv2d):
              self.layer15 = choose_layer(architecture[i], self.layer7.out_channels)
            elif isinstance(self.layer6, nn.Conv2d):
              self.layer15 = choose_layer(architecture[i], self.layer6.out_channels)
            elif isinstance(self.layer5, nn.Conv2d):
              self.layer15 = choose_layer(architecture[i], self.layer5.out_channels)
            elif isinstance(self.layer4, nn.Conv2d):
              self.layer15 = choose_layer(architecture[i], self.layer4.out_channels)
            elif isinstance(self.layer3, nn.Conv2d):
              self.layer15 = choose_layer(architecture[i], self.layer3.out_channels)
            elif isinstance(self.layer2, nn.Conv2d):
              self.layer15 = choose_layer(architecture[i], self.layer2.out_channels)
            elif isinstance(self.layer1, nn.Conv2d):
              self.layer15 = choose_layer(architecture[i], self.layer1.out_channels)
            else:
              self.layer15 = choose_layer(architecture[i])
          else:
            self.layer15 = choose_layer(architecture[i])
        case 15:
          #layer16
          if (architecture[i]>=1 and architecture[i]<=5) or architecture[i]==11: #if it is a convolutional layer or a batchnorm layer, we must set its in_channels value to the out_channels of the last convolutional layer
            if isinstance(self.layer15, nn.Conv2d):
              self.layer16 = choose_layer(architecture[i], self.layer15.out_channels)
            elif isinstance(self.layer14, nn.Conv2d):
              self.layer16 = choose_layer(architecture[i], self.layer14.out_channels)
            elif isinstance(self.layer13, nn.Conv2d):
              self.layer16 = choose_layer(architecture[i], self.layer13.out_channels)
            elif isinstance(self.layer12, nn.Conv2d):
              self.layer16 = choose_layer(architecture[i], self.layer12.out_channels)
            elif isinstance(self.layer11, nn.Conv2d):
              self.layer16 = choose_layer(architecture[i], self.layer11.out_channels)
            elif isinstance(self.layer10, nn.Conv2d):
              self.layer16 = choose_layer(architecture[i], self.layer10.out_channels)
            elif isinstance(self.layer9, nn.Conv2d):
              self.layer16 = choose_layer(architecture[i], self.layer9.out_channels)
            elif isinstance(self.layer8, nn.Conv2d):
              self.layer16 = choose_layer(architecture[i], self.layer8.out_channels)
            elif isinstance(self.layer7, nn.Conv2d):
              self.layer16 = choose_layer(architecture[i], self.layer7.out_channels)
            elif isinstance(self.layer6, nn.Conv2d):
              self.layer16 = choose_layer(architecture[i], self.layer6.out_channels)
            elif isinstance(self.layer5, nn.Conv2d):
              self.layer16 = choose_layer(architecture[i], self.layer5.out_channels)
            elif isinstance(self.layer4, nn.Conv2d):
              self.layer16 = choose_layer(architecture[i], self.layer4.out_channels)
            elif isinstance(self.layer3, nn.Conv2d):
              self.layer16 = choose_layer(architecture[i], self.layer3.out_channels)
            elif isinstance(self.layer2, nn.Conv2d):
              self.layer16 = choose_layer(architecture[i], self.layer2.out_channels)
            elif isinstance(self.layer1, nn.Conv2d):
              self.layer16 = choose_layer(architecture[i], self.layer1.out_channels)
            else:
              self.layer16 = choose_layer(architecture[i])
          else:
            self.layer16 = choose_layer(architecture[i])
        case _:
          print("Error")

    #Define the input size of linear layer
    print("The input size for the first nn.Linear layer is: ", size)

    size2 = int(size/2)
    self.layer17 = nn.Sequential(
        nn.Dropout(p=0.2),
        nn.Linear(in_features=size, out_features=size2),
        nn.Linear(in_features=size2, out_features=2)
    )


  def forward(self, image):
    x = self.layer0(image)
    #print("LAYER 0: ", x.shape)
    x = self.layer1(x)
    #print("LAYER 1: ", x.shape)
    x = self.layer2(x)
    #print("LAYER 2: ", x.shape)
    x = self.layer3(x)
    #print("LAYER 3: ", x.shape)
    x = self.layer4(x)
    #print("LAYER 4: ", x.shape)
    x = self.layer5(x)
    #print("LAYER 5: ", x.shape)
    x = self.layer6(x)
    #print("LAYER 6: ", x.shape)
    x = self.layer7(x)
    #print("LAYER 7: ", x.shape)
    x = self.layer8(x)
    #print("LAYER 8: ", x.shape)
    x = self.layer9(x)
    #print("LAYER 9: ", x.shape)
    x = self.layer10(x)
    #print("LAYER 10: ", x.shape)
    x = self.layer11(x)
    x = self.layer12(x)
    x = self.layer13(x)
    x = self.layer14(x)
    x = self.layer15(x)
    x = self.layer16(x)
    x = self.flatter(x)
    #print("LAYER FLATTER: ", x.shape)
    x = self.layer17(x)
    #print("LAYER 11: ", x.shape)
    return x

#from PIL import Image
#import torchvision.transforms as transforms
#file_path = '/content/drive/MyDrive/LinkToOncoVision/SE4AI/Data/Datasets/FinalDataset/benign/ISIC_0000000.jpg'
#image = image = Image.open(file_path)
##Transform the image in tensor
#transform = transforms.ToTensor()
#img_tensor = transform(image)
#img_tensor = torch.unsqueeze(img_tensor, 0)
#
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#print("Device: ",device)
#img_tensor = img_tensor.to(device)
#model = ConvModel2([2,5,10,9,3,8,11,9,4,6],50*37*4).to(device)
#print(model)
#model.forward(img_tensor)